In [1]:
tissues_with_atac = ['CHO','Brain','Liver','Lung','Kidney','Spleen','BMDMwt','BMDM1hKLA']

## Parameters specific to where your folders are and your data
parameter_file = 'params/params.yaml'
import yaml
import sys

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
annotation=doc['annotation']


tss_annotation = doc['tss_annotation']

import os
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm
### notebook specific configuration ###
from os.path import basename
#mpl.style.use('ggplot')
mpl.style.use('fivethirtyeight')
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

from Homer import *

print('Number of tissues: ',len(tissues))

('Number of tissues: ', 15)


In [2]:
from create_output import *

In [3]:
txn_f = "Results/tss_annotation/txn_df_02_tissues.p"
#expr_f = "Results/merged/samples.merge.peaksexpression.log10"
expr_f = "Results/merged/tissues.merge.peaksexpression.log10"
anno_f = "Results/tss_annotation_peaks/all_peaks_txn_df_02_maxinfo.tsv"
rank_func = rank_median_samples
atac_f = "Results/ATAC_results/CHO/ATAC_ppr.naive_overlap.narrowPeak.sort"
tss_f = "/data/isshamie/genome/picr_final/mRNA_final.peak"#tss_annotation
all_atac = "Results/ATAC_results/ATAC_merge.bed"



In [ ]:
%%time
# bed_df, meta_df = construct_peakID(txn_anno_f, expr_f, anno_f, rank_func,
#                      atac_f=None, col_name=None)

bed_df, meta_df = construct_all_peaks(txn_f, expr_f, anno_f, rank_func, tss_f, atac_f,
                        all_atac=all_atac, out_f="Results/output/TSS1")

26543it [07:34, 22.50it/s]
11429it [04:02, 30.53it/s]

In [5]:
bed_df.head()

,Chr,Start,End,ID,Stat,Strand
p0@SERHL_gene_1_2_asmbl_2,picr_0,175,177,p0@SERHL_gene_1_2_asmbl_2,0.00000,+
p0@SERHL_gene_1_4_asmbl_11,picr_0,18148,18150,p0@SERHL_gene_1_4_asmbl_11,0.00000,+
p0@SERHL_gene_1_4_asmbl_12,picr_0,20786,20788,p0@SERHL_gene_1_4_asmbl_12,0.00000,+
p1@SERHL_gene_1_4_asmbl_14,picr_0,20916,21066,p1@SERHL_gene_1_4_asmbl_14,2.05576,+
p0@SERHL_gene_1_4_asmbl_15,picr_0,22503,22505,p0@SERHL_gene_1_4_asmbl_15,0.00000,+


In [6]:
meta_df.head()

,Tissues,cs,CHO ATAC Region,ID,Gene,Gene ID,Transcript,Is Experimental,has ATAC
p0@SERHL_gene_1_2_asmbl_2,,,,,SERHL,gene_1_2,asmbl_2,0,0
p0@SERHL_gene_1_4_asmbl_11,,,,,SERHL,gene_1_4,asmbl_11,0,0
p0@SERHL_gene_1_4_asmbl_12,,,,,SERHL,gene_1_4,asmbl_12,0,0
p1@SERHL_gene_1_4_asmbl_14,"Heart,BMDMwt,Spleen,FemaleReproductive,CHO,Bra...",0b11,,55675,SERHL,gene_1_4,asmbl_14,1,0
p0@SERHL_gene_1_4_asmbl_15,,,,,SERHL,gene_1_4,asmbl_15,0,0


In [57]:
sys.path.append("/home/isshamie/software/homebrew/parallel_functions/")

In [58]:
import parallel_functions as pf

In [ ]:
pf.parallel_df(out_peaks_df,func=par_atac_region,func_args=(atac_f, all_atac),num_processes=16)

Process PoolWorker-158:
Traceback (most recent call last):
Process PoolWorker-148:
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Process PoolWorker-151:
    self.run()
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
Process PoolWorker-149:
Process PoolWorker-155:
Traceback (most recent call last):
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/pool.py", line 113, in worker
Process PoolWorker-146:
Traceback (most recent call last):
Process PoolWorker-150:
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Process PoolWorker-145:
    result = (True, func(*args, **kwds))
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
    self.run()
Traceback (most rece

  File "/home/isshamie/software/homebrew/parallel_functions/parallel_functions.py", line 18, in multi_run_wrapper
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/isshamie/software/homebrew/parallel_functions/parallel_functions.py", line 18, in multi_run_wrapper
    return map(*args)
    return curr_func(args[1], *args[2])
    result = (True, func(*args, **kwds))
    self.run()
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/pool.py", line 65, in mapstar
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/pool.py", line 113, in worker
    return curr_func(args[1], *args[2])
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/pool.py", line 65, in mapstar
    return map(*args)
    self._target(*self._args, **self._kwargs)
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/pool.py", line 65, in mapstar
  File "/home/isshamie/software/anac

    result = _comp_method_OBJECT_ARRAY(op, x, y)
    self.obj._set_value(*key, takeable=self._takeable)
  File "/home/isshamie/software/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py", line 189, in __setitem__
    self.obj._set_value(*key, takeable=self._takeable)
    all_df.at[ind, "has ATAC"] = 1
    curr = all_df[all_df["Chr"] == val["Chr"]]
  File "/home/isshamie/software/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py", line 2587, in _set_value
    self.obj._set_value(*key, takeable=self._takeable)
    self.obj._set_value(*key, takeable=self._takeable)
    data = self._data.copy(deep=deep)
  File "/data/isshamie/TSS/Analysis/src/create_output.py", line 331, in par_atac_region
  File "/home/isshamie/software/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py", line 189, in __setitem__
  File "/data/isshamie/TSS/Analysis/src/create_output.py", line 328, in par_atac_region
    res = na_op(values, other)
    return curr_func(args[1], *args[2])
  Fi

    values = values.copy()
  File "/home/isshamie/software/anaconda2/lib/python2.7/site-packages/pandas/core/indexes/base.py", line 3064, in get_loc
  File "/home/isshamie/software/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py", line 5110, in copy
  File "/home/isshamie/software/anaconda2/lib/python2.7/site-packages/pandas/util/_decorators.py", line 187, in wrapper
KeyboardInterrupt
  File "/home/isshamie/software/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py", line 376, in _setitem_with_indexer
    self.obj[item] = s
  File "/home/isshamie/software/anaconda2/lib/python2.7/site-packages/pandas/core/indexes/base.py", line 3064, in get_loc
    data = self._data.copy(deep=deep)
  File "/home/isshamie/software/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py", line 543, in setter
    result = libops.scalar_compare(x, y, op)
    self._setitem_with_indexer(indexer, value)
  File "/home/isshamie/software/anaconda2/lib/python2.7/site-packages/pandas/

KeyboardInterrupt
  File "pandas/_libs/hashtable_class_helper.pxi", line 1554, in pandas._libs.hashtable.PyObjectHashTable.lookup
    stream.send_multipart(to_send, copy=copy)
  File "/home/isshamie/software/anaconda2/lib/python2.7/site-packages/ipykernel/iostream.py", line 249, in send_multipart
  File "/home/isshamie/software/anaconda2/lib/python2.7/site-packages/numpy/core/numeric.py", line 463, in asarray
    def asarray(a, dtype=None, order=None):
    return self.io_thread.send_multipart(*args, **kwargs)
KeyboardInterrupt
  File "/home/isshamie/software/anaconda2/lib/python2.7/site-packages/ipykernel/iostream.py", line 199, in send_multipart
    self.schedule(lambda : self._really_send(*args, **kwargs))
  File "/home/isshamie/software/anaconda2/lib/python2.7/site-packages/ipykernel/iostream.py", line 192, in schedule
    f()
  File "/home/isshamie/software/anaconda2/lib/python2.7/site-packages/ipykernel/iostream.py", line 199, in <lambda>
    self.schedule(lambda : self._really_se

...

  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/pool.py", line 102, in worker
    self._target(*self._args, **self._kwargs)
    self._target(*self._args, **self._kwargs)
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/pool.py", line 102, in worker
    self._target(*self._args, **self._kwargs)
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/pool.py", line 102, in worker
    self._target(*self._args, **self._kwargs)
    self._target(*self._args, **self._kwargs)
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/pool.py", line 102, in worker
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/pool.py", line 102, in worker
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/pool.py", line 102, in worker
    self._target(*self._args, **self._kwargs)
    task = get()
    self._target(*self._args, **self._kwargs)
  File "/home/isshamie/software/anaconda2/lib/pyt

Process PoolWorker-267:
Process PoolWorker-268:
Process PoolWorker-259:
Process PoolWorker-258:
Traceback (most recent call last):
Process PoolWorker-273:
Traceback (most recent call last):
Traceback (most recent call last):
Process PoolWorker-272:
Traceback (most recent call last):
Traceback (most recent call last):
Process PoolWorker-262:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/home/isshamie/software/anaconda2/lib/python2.

  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/pool.py", line 102, in worker
    task = get()
    task = get()
    task = get()
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/pool.py", line 102, in worker
    task = get()
    task = get()
    task = get()
    task = get()
    task = get()
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/queues.py", line 376, in get
    task = get()
    task = get()
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/queues.py", line 376, in get
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/queues.py", line 376, in get
    task = get()
    task = get()
    task = get()
    task = get()
    task = get()
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/queues.py", line 376, in get
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/queues.py", line 376, in get
  File "/home/isshamie/software/anac

  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
    self.run()
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiproces

    racquire()
KeyboardInterrupt
    racquire()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
    racquire()
    racquire()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
    self._target(*self._args, **self._kwargs)
KeyboardInterrupt
KeyboardInterrupt
Traceback (most recent call last):
    racquire()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/pool.py", line 102, in worker
KeyboardInterrupt
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    task = get()
    self.run()
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/queues.py", line 376, in get
  File "/home/isshamie/software/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._

-----------------------------

-----------------------------

-----------------------------

-----------------------------

-----------------------------